In [1]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [1]:
# Instalamos Spark  
!pip install pyspark py4j
!pip install pydeequ

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 53.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=8ab36d143b0b07325c06ee1cd10e321e80dd4c51fd4454b75d75f263b84160aa
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
# solo google colab
import io
from google.colab import files

# Cargando data Google Colab
uploaded = files.upload()

Saving ._SUCCESS.crc to ._SUCCESS.crc
Saving .part-00000-28574fd8-2cee-454c-a2f1-e99bd789f164-c000.snappy.parquet.crc to .part-00000-28574fd8-2cee-454c-a2f1-e99bd789f164-c000.snappy.parquet.crc
Saving _SUCCESS to _SUCCESS
Saving part-00000-28574fd8-2cee-454c-a2f1-e99bd789f164-c000.snappy.parquet to part-00000-28574fd8-2cee-454c-a2f1-e99bd789f164-c000.snappy.parquet


In [4]:
from pyspark.sql import SparkSession, Row
import pydeequ 


spark = (SparkSession
    .builder
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())

Please set env variable SPARK_VERSION


In [8]:
from pyspark.sql.types import StructField, StructType, StringType, TimestampType, IntegerType,BooleanType, FloatType,TimestampType,DateType
schema = StructType([
        StructField("province",StringType()),
    StructField('city', StringType()),
    StructField("patient_id",IntegerType()),
    StructField("sex",StringType()),
    StructField("age",IntegerType()),
    StructField("country",StringType()),
    StructField("infection_case",StringType()),
    StructField("symptom_onset_date",DateType()),
    StructField("confirmed_date",DateType()),
    StructField("state",StringType()),
    StructField("Region_Code",IntegerType()),
    StructField('latitude', FloatType()),
    StructField('longitude', FloatType()),
    StructField('elementary_school_count', IntegerType()),
    StructField('kindergarten_count', IntegerType()),
    StructField('university_count', IntegerType()),
    StructField('nursing_home_count', IntegerType()),
    StructField("code",IntegerType()),
    StructField("date",DateType()),
    StructField("avg_temp",FloatType()),
    StructField("min_temp",FloatType()),
    StructField("max_temp",FloatType()),
    StructField("precipitation",FloatType()),
    StructField("max_wind_speed",FloatType()),
    StructField("most_wind_direction",IntegerType()),
    StructField("avg_relative_humidity",FloatType())
])


df =  spark.read.schema(schema).parquet('coronavirusTable/')
df.show(5)

+--------+-------+----------+----+---+-------+---------------+------------------+--------------+--------+-----------+---------+---------+-----------------------+------------------+----------------+------------------+-----+----------+--------+--------+--------+-------------+--------------+-------------------+---------------------+
|province|   city|patient_id| sex|age|country| infection_case|symptom_onset_date|confirmed_date|   state|Region_Code| latitude|longitude|elementary_school_count|kindergarten_count|university_count|nursing_home_count| code|      date|avg_temp|min_temp|max_temp|precipitation|max_wind_speed|most_wind_direction|avg_relative_humidity|
+--------+-------+----------+----+---+-------+---------------+------------------+--------------+--------+-----------+---------+---------+-----------------------+------------------+----------------+------------------+-----+----------+--------+--------+--------+-------------+--------------+-------------------+---------------------+
|   

In [11]:
from pydeequ.checks import *
from pydeequ.verification import *

# En local window aparece el error TypeError: 'JavaPackage' object is not callable
check = Check(spark, CheckLevel.Warning, "Review Check") 

checkResult = (VerificationSuite(spark)
    .onData(df)
    .addCheck(check.hasMax("age", lambda x: x == 30)  
        .isNonNegative("age"))
    .run()
)
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+------------+-----------+------------+----------------------------------------------------------------------------------------------------------+-----------------+------------------+
|check       |check_level|check_status|constraint                                                                                                |constraint_status|constraint_message|
+------------+-----------+------------+----------------------------------------------------------------------------------------------------------+-----------------+------------------+
|Review Check|Warning    |Success     |MaximumConstraint(Maximum(age,None))                                                                      |Success          |                  |
|Review Check|Warning    |Success     |ComplianceConstraint(Compliance(age is non-negative,COALESCE(CAST(age AS DECIMAL(20,10)), 0.0) >= 0,None))|Success          |                  |
+------------+-----------+------------+-----------------------------------------